<a href="https://colab.research.google.com/github/LIKHITH-22/DERMO/blob/main/FED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf /content/Federated_Learning
!mkdir /content/Federated_Learning


In [2]:
!touch /content/Federated_Learning/server.py
!touch /content/Federated_Learning/client.py
!touch /content/Federated_Learning/model.py
!touch /content/Federated_Learning/utils.py



In [3]:
!unzip -q /content/train.zip -d /content/Federated_Learning/


unzip:  cannot find or open /content/train.zip, /content/train.zip.zip or /content/train.zip.ZIP.


In [4]:
from google.colab import files
from zipfile import ZipFile

# Upload the ZIP file
uploaded = files.upload()

# Extract the uploaded file (Assuming it's named 'train.zip')
zip_path = "/content/train.zip"  # Change this if the file name is different
with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/Federated_Learning/")  # Extract into correct location

print("✅ Extraction Complete! Data is in /content/Federated_Learning/train")


Saving train.zip to train.zip
✅ Extraction Complete! Data is in /content/Federated_Learning/train


In [5]:
import tensorflow as tf
import os

def load_data(data_dir="/content/Federated_Learning/train/train", batch_size=32, img_size=(224, 224)):
    """
    Loads dataset from the specified directory and applies normalization.

    Args:
        data_dir (str): Path to dataset (with the extra "train" folder).
        batch_size (int): Batch size.
        img_size (tuple): Target image size.

    Returns:
        train_dataset (tf.data.Dataset): Preprocessed training dataset.
        class_names (list): Class labels.
    """
    print("✅ Checking dataset contents:", os.listdir(data_dir))  # Should show ['NonPsor', 'pSOR']

    train_dataset = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        labels="inferred",  # Infer labels from folder names
        label_mode="categorical",  # Use categorical labels for classification
        image_size=img_size,
        batch_size=batch_size,
        shuffle=True
    )

    # Get class names
    class_names = train_dataset.class_names
    print("📌 Classes found:", class_names)

    # Normalize images (0-1 range)
    normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)
    train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))

    return train_dataset, class_names


['pSOR', 'NonPsor']


In [13]:
import sys
sys.path.append("/content/Federated_Learning")


In [15]:
!cp /content/Federated_Learning/utils.py /content/Federated_Learning/utils.py


cp: '/content/Federated_Learning/utils.py' and '/content/Federated_Learning/utils.py' are the same file


In [16]:
import importlib
import utils
importlib.reload(utils)  # Reload after changes
train_dataset, class_names = utils.load_data()
print("✅ Final class names:", class_names)



✅ Checking dataset contents: ['pSOR', 'NonPsor']
Found 1334 files belonging to 2 classes.
📌 Classes found: ['NonPsor', 'pSOR']
✅ Final class names: ['NonPsor', 'pSOR']


In [8]:
!mv "/content/drive/MyDrive/train" "/content/drive/MyDrive/Federated_Learning/"


In [12]:
!cp /content/drive/MyDrive/Federated_Learning/utils.py /content/utils.py
!cp /content/drive/MyDrive/Federated_Learning/server.py /content/server.py
!cp /content/drive/MyDrive/Federated_Learning/client.py /content/client.py
!cp /content/drive/MyDrive/Federated_Learning/model.py /content/model.py


In [24]:
import model

num_classes = 2  # Since we have 'pSOR' and 'NonPsor'
model_type = "efficientnet"  # Or "resnet"

cnn_model = model.create_model(num_classes, model_type)
cnn_model.summary()  # Check model structure


165234480/165234480 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_3 (Rescaling)   │ (None, 224, 224, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, 224, 224, 3)    │              7 │ rescaling_3[0][0]      │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_4 (Rescaling)   │ (None, 224, 224, 3)    │              0 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 225, 225, 3)    │              0 │ rescaling_4[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 112, 112, 56)   │          1,512 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 112, 112, 56)   │            224 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 112, 112, 56)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 112, 112, 56)   │            504 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 112, 112, 56)   │            224 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 112, 112, 56)   │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 56)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 56)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 14)       │            798 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_expand    

 Total params: 42,272,145 (161.26 MB)

 Trainable params: 1,312,002 (5.00 MB)

 Non-trainable params: 40,960,143 (156.25 MB)

In [26]:
%%writefile /content/server_api.py
from flask import Flask, request, jsonify
import tensorflow as tf
import numpy as np
from model import create_model, load_model, save_model

app = Flask(__name__)

# Load existing model or create a new one
try:
    model = load_model("server_model.h5")
    print("✅ Loaded existing model.")
except:
    model = create_model(num_classes=2)
    print("⚡ Created new model.")

@app.route("/predict", methods=["POST"])
def predict():
    """Receives an image from the client and makes a prediction."""
    data = request.json
    image = np.array(data["image"], dtype=np.float32)
    image = tf.image.resize(image, (224, 224)) / 255.0
    image = tf.expand_dims(image, axis=0)

    prediction = model.predict(image)
    pred_class = np.argmax(prediction)
    confidence = float(np.max(prediction))

    return jsonify({"pred_class": int(pred_class), "confidence": confidence})

@app.route("/train", methods=["POST"])
def train():
    """Retrains model on incorrectly predicted samples."""
    data = request.json
    image = np.array(data["image"], dtype=np.float32)
    correct_label = int(data["correct_label"])

    image = tf.image.resize(image, (224, 224)) / 255.0
    image = tf.expand_dims(image, axis=0)

    label = np.array([[correct_label]], dtype=np.int32)

    model.fit(image, label, epochs=1, verbose=1)
    save_model(model, "server_model.h5")

    return jsonify({"message": "Model retrained successfully!"})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


Writing /content/server_api.py


In [ ]:
!python server_api.py &


2025-02-23 08:28:07.193525: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-23 08:28:07.248622: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-23 08:28:07.263955: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-23 08:28:09.821581: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚡ Created new model.
 * Serving Flask app 'server_api'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.2:5000
Press CTRL+C to quit
